In [1]:
from z3 import *

from itertools import combinations
from z3 import *
from utils import *
import math


In [2]:
file_name='inst01.dat'
file = open('./Instances/'+file_name, 'r')
splitted_file = file.read().split('\n')
m = int(splitted_file[0])
n = int(splitted_file[1])
cpt = list(map(int, splitted_file[2].split(' ')))
sz = list(map(int, splitted_file[3].split(' ')))
D = [list(map(int, line.strip().split(' '))) for line in splitted_file[4:(n+5)]]
print(D)

[[0, 3, 4, 5, 6, 6, 2], [3, 0, 1, 4, 5, 7, 3], [4, 1, 0, 5, 6, 6, 4], [4, 4, 5, 0, 3, 3, 2], [6, 7, 8, 3, 0, 2, 4], [6, 7, 8, 3, 2, 0, 4], [2, 3, 4, 3, 4, 4, 0]]


In [3]:
def at_least_one_np(bool_vars):
    return Or(bool_vars)

def at_most_one_np(bool_vars, name = ""):
    return And([Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)])

def exactly_one_np(bool_vars, name = ""):
    return And(at_least_one_np(bool_vars), at_most_one_np(bool_vars, name))

In [4]:
def at_least_one_seq(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_seq(bool_vars, name):
    constraints = []
    n = len(bool_vars)
    s = [Bool(f"s_{name}_{i}") for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2])))
    for i in range(1, n - 1):
        constraints.append(Or(Not(bool_vars[i]), s[i]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1])))
        constraints.append(Or(Not(s[i-1]), s[i]))
    return And(constraints)

def exactly_one_seq(bool_vars, name):
    return And(at_least_one_seq(bool_vars), at_most_one_seq(bool_vars, name))

In [5]:
def at_least_k_seq(bool_vars, k, name):
    return at_most_k_seq([Not(var) for var in bool_vars], len(bool_vars)-k, name)

def at_most_k_seq(bool_vars, k, name):
    constraints = []
    n = len(bool_vars)
    s = [[Bool(f"s_{name}_{i}_{j}") for j in range(k)] for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0][0]))
    constraints += [Not(s[0][j]) for j in range(1, k)]
    for i in range(1, n-1):
        constraints.append(Or(Not(bool_vars[i]), s[i][0]))
        constraints.append(Or(Not(s[i-1][0]), s[i][0]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][k-1])))
        for j in range(1, k):
            constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][j-1]), s[i][j]))
            constraints.append(Or(Not(s[i-1][j]), s[i][j]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2][k-1])))   
    return And(constraints)

def exactly_k_seq(bool_vars, k, name):
    return And(at_most_k_seq(bool_vars, k, name+'1'), at_least_k_seq(bool_vars, k, name))

In [6]:
def print_items_for_each_courier(couriers_items, items_predecessors, sz, cpt):
    # Creare un dizionario per mappare ogni corriere ai suoi pacchi
    couriers_dict = {}
    for i, j in couriers_items:
        if i not in couriers_dict:
            couriers_dict[i] = []
        couriers_dict[i].append(j)
    #print('cour_dict', couriers_dict)

    n=0
    for value in couriers_dict.values():
        n = n+len(value)
    #print(n)


    # Creare un altro dizionario per mappare ogni pacco al suo predecessore
    
    predecessors_dict = {i: j for j, i in items_predecessors}
    #print('pred_dict', predecessors_dict)
    
    # Per ogni corriere, seguire la catena di predecessori per ottenere l'ordine corretto dei pacchi
    for courier, items in couriers_dict.items():
        print(f"Corriere {courier}: ", end='')
        item = n + courier
        order = [item]
        while item in predecessors_dict and predecessors_dict[item] not in order:
            item = predecessors_dict[item]
            order.append(item)
        item = predecessors_dict[item]
        order.append(item)
        print(" -> ".join(map(str, order)), end='   ')
        print('Peso trasportato: ', end='')
        weight=0
        for i in couriers_dict[courier]:
            weight += sz[i]

        print(weight, '/', cpt[courier])

In [7]:
def max_z3(vec):
  m = vec[0]
  for value in vec[1:]:
    m = If(value > m, value, m)
  return m

In [8]:
'''n packs
   m couriers
   D matix int(n+1,m+1)
   sz array of item's sizes (len = n)
   cpt array of couriers' capacities (len = m)
'''
def mcp(n, m, D, sz, cpt):
   
   '''
      pred matrix  
   '''
   #pred[i][j] = true if the j-th item is the predecessor of the i-th item. 
   #The n+i column in the matrix is the starting point of the i-th courier. The n+i row in the matrix is the ending point of the i-th courier
   pred = [[Bool(f"pred({i})_{j}")for j in range(n+m)]for i in range(n+m)]

   #create Solver
   solv= Solver()

   #Each item/ending point has exactly one predecessor and each item/starting point is predecessor of exactly one other item. 
   #i=0 to i=n+m-1
   #j=0 to j=n+m-1 
   for i in range(n+m):
      col_i = []
      for j in range(n+m):
         col_i += [pred[j][i]]

      solv.add(exactly_one_seq(col_i, f"PC_{i}"))
      solv.add(exactly_one_seq(pred[i], f"PR_{i}"))

   
   '''
      vehicle matrix 
      col = packs
      rows = courriers 
      each rows represent the list of packs foreach courrier
   ''' 
   #create a [m x (n+m)] matrix v. v[i][j]==True if the i-th courier takes the j-th item (or starting/ending point)
   v=[[Bool(f"vehicle({i})_{j}")for j in range(n+m)]for i in range(m)]

   #each pack have to be carried by exactly one courrier
   for j in range(n):
      solv.add(exactly_one_seq([v[i][j] for i in range(m)], f"v_{j}"))

   for j in range(m):
      for i in range(m):
         if j==i:
            solv.add(v[i][n+j])
         else:
            solv.add(Not(v[i][n+j]))
   
   #Constraint coerenza courier - predecessore/successore
   for c in range(m):
      solv.add(And(  [Implies(And([ v[c][i], pred[i][j]] ) , v[c][j]) for i in range(n+m) for j in range(n+m)]  ))

   
   #Constraint peso
   for c in range(m):
      solv.add(Sum([If(v[c][i], 1, 0)*sz[i] for i in range(n)]) <= cpt[c])
   
   '''
      no_loops 
   '''
   no_loops = IntVector('no_loops', n)
   solv.add(And([Implies(pred[i][j], no_loops[i]>no_loops[j]) for i in range(n) for j in range(n)]))
   #solv.add(no_loops[0]==1)

   #Distance
   dist_vector = IntVector('dist_vector', m)
   for courier in range(m):
      starting_point = [If(pred[item][n+courier], 1, 0)*D[n][item] for item in range(n)]
      ending_point = [If(pred[n+courier][item], 1, 0)*D[item][n] for item in range(n)]
      mid_points = [If(And(v[courier][i], pred[i][j]), 1, 0)*D[j][i] for i in range(n) for j in range(n)]
      distance_of_this_path = Sum(starting_point + mid_points + ending_point)
      solv.add(dist_vector[courier] == distance_of_this_path)
   max_dist = Int('max_dist')
   max_dist = max_z3(dist_vector)
   
   solv.check()
   mod = solv.model()

   
   item_pred, cour_item = [(i,j) for j in range(n+m) for i in range(n+m) if mod.evaluate(pred[i][j])], [(i, j) for j in range(n) for i in range(m) if mod.evaluate(v[i][j])]
   vector_distance = [mod.evaluate(dist_vector[i]) for i in range(m)]

   return item_pred, cour_item, max_dist
   
# -----------------------------------------------------------------------------

item_pred, cour_item, _ = mcp(n,m,D,sz,cpt)

print("Output of the model:", item_pred, cour_item)
print("\nPrint of the solutions:\n")
print_items_for_each_courier( cour_item, item_pred, sz, cpt)

Output of the model: [(2, 0), (0, 1), (4, 2), (5, 3), (6, 4), (7, 5), (1, 6), (3, 7)] [(0, 0), (0, 1), (0, 2), (1, 3), (0, 4), (1, 5)]

Print of the solutions:

Corriere 0: 6 -> 1 -> 0 -> 2 -> 4 -> 6   Peso trasportato: 15 / 15
Corriere 1: 7 -> 3 -> 5 -> 7   Peso trasportato: 9 / 10


In [ ]:
def solver_builder(n, m, D, sz, cpt):
   
   '''
      pred matrix  
   '''
   #pred[i][j] = true if the j-th item is the predecessor of the i-th item. 
   #The n+i column in the matrix is the starting point of the i-th courier. The n+i row in the matrix is the ending point of the i-th courier
   pred = [[Bool(f"pred({i})_{j}")for j in range(n+m)]for i in range(n+m)]

   #create Solver
   solv= Solver()

   #Each item/ending point has exactly one predecessor and each item/starting point is predecessor of exactly one other item. 
   #i=0 to i=n+m-1
   #j=0 to j=n+m-1 
   for i in range(n+m):
      col_i = []
      for j in range(n+m):
         col_i += [pred[j][i]]

      solv.add(exactly_one_seq(col_i, f"PC_{i}"))
      solv.add(exactly_one_seq(pred[i], f"PR_{i}"))

   
   '''
      vehicle matrix 
      col = packs
      rows = courriers 
      each rows represent the list of packs foreach courrier
   ''' 
   #create a [m x (n+m)] matrix v. v[i][j]==True if the i-th courier takes the j-th item (or starting/ending point)
   v=[[Bool(f"vehicle({i})_{j}")for j in range(n+m)]for i in range(m)]

   #each pack have to be carried by exactly one courrier
   for j in range(n):
      solv.add(exactly_one_seq([v[i][j] for i in range(m)], f"v_{j}"))

   for j in range(m):
      for i in range(m):
         if j==i:
            solv.add(v[i][n+j])
         else:
            solv.add(Not(v[i][n+j]))
   
   #Constraint coerenza courier - predecessore/successore
   for c in range(m):
      solv.add(And(  [Implies(And([ v[c][i], pred[i][j]] ) , v[c][j]) for i in range(n+m) for j in range(n+m)]  ))

   
   #Constraint peso
   for c in range(m):
      solv.add(Sum([If(v[c][i], 1, 0)*sz[i] for i in range(n)]) <= cpt[c])
   
   '''
      no_loops 
   '''
   no_loops = IntVector('no_loops', n)
   solv.add(And([Implies(pred[i][j], no_loops[i]>no_loops[j]) for i in range(n) for j in range(n)]))
   #solv.add(no_loops[0]==1)

   #Distance
   dist_vector = IntVector('dist_vector', m)
   for courier in range(m):
      starting_point = [If(pred[item][n+courier], 1, 0)*D[n][item] for item in range(n)]
      ending_point = [If(pred[n+courier][item], 1, 0)*D[item][n] for item in range(n)]
      mid_points = [If(And(v[courier][i], pred[i][j]), 1, 0)*D[j][i] for i in range(n) for j in range(n)]
      distance_of_this_path = Sum(starting_point + mid_points + ending_point)
      solv.add(dist_vector[courier] == distance_of_this_path)
   max_dist = Int('max_dist')
   max_dist = max_z3(dist_vector)
   
   solv.check()
   mod = solv.model()

   
   item_pred, cour_item = [(i,j) for j in range(n+m) for i in range(n+m) if mod.evaluate(pred[i][j])], [(i, j) for j in range(n) for i in range(m) if mod.evaluate(v[i][j])]
   vector_distance = [mod.evaluate(dist_vector[i]) for i in range(m)]

   return item_pred, cour_item, max_dist
   

In [ ]:
# Given formula F, find the model the maximizes the value of X 
# using at-most M iterations.
def max(F, X, M):
    s = Solver()
    s.add(F)
    last_model  = None
    i = 0
    while True:
        r = s.check()
        if r == unsat:
            if last_model != None:
                return last_model
            else:
                return unsat
        if r == unknown:
            raise Z3Exception("failed")
        last_model = s.model()
        s.add(X > last_model[X])
        i = i + 1
        if (i > M):
            raise Z3Exception("maximum not found, maximum number of iterations was reached")

x, y = Ints('x y')
F = [x > 0, x < 10, x == 2*y]
print max(F, x, 10000)